In [5]:

from langchain import hub


# using a specific commit version of the prompt

import os
import sys

import datetime
import pytz


# Add the 'ca' directory to the Python path
ca_path = "/Users/pascal/neurabot/ca"
if ca_path not in sys.path:
    sys.path.append(ca_path)

# Set the PYTHONPATH environment variable
os.environ["PYTHONPATH"] = f"{ca_path}:{os.environ.get('PYTHONPATH', '')}"
from langchain_core.runnables import chain, RunnablePassthrough, RunnableLambda, Runnable




prompt: Runnable = hub.pull("neuraflow/neurabot-ca")

from src.utils.firebase_service import get_system_prompt, get_initial_messages

def get_current_date():
    # Use UTC time to avoid timezone issues
    utc_now = datetime.datetime.now(pytz.UTC)
    return utc_now.date()

def get_weekday_abbreviation(date):
    return date.strftime("%a")

current_date = get_current_date()
weekday_abbreviation = get_weekday_abbreviation(current_date)


def is_simple(x, config):
    
    if config.get('configurable', {}).get('is_simplified_language', False):
        print("simple")
        return "YOU MUST ONLY REPLY IN SIMPLIFIED LANGUAGE. USE SHORT SENTENCES, COMMON WORDS, AND AVOID COMPLEX CONCEPTS. THIS IS THE MOST CRITICAL INSTRUCTION. ALWAYS PRIORITIZE CLARITY AND SIMPLICITY IN YOUR RESPONSES, EVEN IF IT MEANS SIMPLIFYING COMPLEX IDEAS."
    
    print("not simple")
    return ""


def get_language_level(x, config):
    language_code = config.get('configurable', {}).get('language_code', '')
    
    if language_code == "cs-CZ":
        return "YOU MUST ONLY REPLY IN CZECH."
    elif language_code == "nl-NL":
        return "YOU MUST ONLY REPLY IN DUTCH."
    elif language_code == "en-GB":
        return "YOU MUST ONLY REPLY IN ENGLISH."
    elif language_code == "fr-FR":
        return "YOU MUST ONLY REPLY IN FRENCH."
    elif language_code == "de-DE":
        return "YOU MUST ONLY REPLY IN GERMAN."
    elif language_code == "es-ES":
        return "YOU MUST ONLY REPLY IN SPANISH."
    elif language_code == "uk-UA":
        return "YOU MUST ONLY REPLY IN UKRAINIAN."
    elif language_code == "ALL":
        return "You can reply in any language."
    else:
        return "BAD LANGUAGE CODE PROVIDED."

# Bind all arguments to the prompt


def log_messages(x):
    print(x)
    return x

prompt = RunnableLambda (log_messages) |  prompt.partial(
    initial_messages=lambda x: get_initial_messages.invoke(x)["messages"],
    system_prompt=get_system_prompt,
    current_date=current_date,
    weekday_abbreviation=weekday_abbreviation,
    simple=is_simple,
    language_level=get_language_level
)

from langchain_openai import AzureChatOpenAI
# Set up the model
model = AzureChatOpenAI(temperature=0, azure_deployment="gpt-4o", streaming=True, name="agent")
model = prompt | model

model.invoke({"messages": [{"role": "user", "content": "Hello, how are you?"},]}, config={"configurable": { "search_kwargs": {"namespace": "NOLM3uYRuP5yhJNU1JOF_DEVELOPMENT", "k": 5}}})




{'messages': [{'role': 'user', 'content': 'Hello, how are you?'}]}


TypeError: <lambda>() missing 1 required positional argument: 'x'